In [27]:
import pandas as pd
import numpy as np
import yfinance
import datetime

from optionalyzer.blackscholes import BlackScholes
from optionalyzer.options import Call, Put
from optionalyzer.chart import PayoffChart

import plotly.io as pio
pio.renderers.default = "notebook_connected"
pio.templates.default = "plotly_dark"

# `BlackScholes`

This class implements the Black-Scholes model for pricing European options. The class has methods to calculate option prices and greeks. It can also be used to calculate implied volatility given the market price of an option. Initialization of the class does not require any parameter.

In [2]:
bs = BlackScholes()

## Downloading the Optiona Chain

We'll be using `yfinance` to download the option chain of a stock. We'll be using the `AAPL` stock for this example.


In [3]:
apple = yfinance.Ticker("AAPL")

#Load at the money options
expiry_date = "2023-04-21"
options = apple.option_chain(date=expiry_date)
calls = options.calls
puts = options.puts

spot_price = apple.history(period="1d")['Close'][0]
print(spot_price)

#At the money options
calls = calls[((calls['strike'] - spot_price) < 20) & ((calls['strike'] - spot_price) > -20)]
puts = puts[((puts['strike'] - spot_price) < 20) & ((puts['strike'] - spot_price) > -20)]


129.6199951171875


In [4]:
calls = calls[["lastTradeDate", "strike", "lastPrice", "percentChange"]]
puts = puts[["lastTradeDate", "strike", "lastPrice", "percentChange"]]

In [5]:
display(calls)
display(puts)

,lastTradeDate,strike,lastPrice,percentChange
11,2023-01-06 20:43:20+00:00,110.0,23.87,18.168316
12,2023-01-06 18:00:03+00:00,115.0,18.41,11.575757
13,2023-01-06 20:57:59+00:00,120.0,15.85,21.455938
14,2023-01-06 20:54:27+00:00,125.0,12.60,26.000006
15,2023-01-06 20:59:47+00:00,130.0,9.70,27.296587
16,2023-01-06 20:54:41+00:00,135.0,7.18,30.545452
17,2023-01-06 20:59:41+00:00,140.0,5.04,30.569950
18,2023-01-06 20:56:29+00:00,145.0,3.45,31.178703


,lastTradeDate,strike,lastPrice,percentChange
11,2023-01-06 20:59:53+00:00,110.0,2.75,-27.631578
12,2023-01-06 20:57:54+00:00,115.0,3.77,-25.787401
13,2023-01-06 20:55:22+00:00,120.0,5.00,-25.595236
14,2023-01-06 20:57:36+00:00,125.0,6.70,-23.428574
15,2023-01-06 20:59:50+00:00,130.0,8.80,-21.777777
16,2023-01-06 20:58:20+00:00,135.0,11.25,-20.212769
17,2023-01-06 19:16:48+00:00,140.0,14.42,-17.458496
18,2023-01-06 19:45:09+00:00,145.0,17.47,-14.110131


In [6]:
strikes = calls['strike'].values
call_prices = calls['lastPrice'].values
put_prices = puts['lastPrice'].values

In [7]:
today = datetime.datetime.today().strftime("%Y-%m-%d")
today

'2023-01-08'

## Calculating Implied Volatility

First, we'll calculate the implied volatility of the option chain. We'll be using the `implied_volatility` method of the `BlackScholes` class.

In [8]:
id_ = 1
S = spot_price
K = strikes[id_]
r = 0.0375
tau = (
    datetime.datetime.strptime(expiry_date, "%Y-%m-%d")
    - datetime.datetime.strptime(today, "%Y-%m-%d")
).days / 365
option_type = "call"
price = call_prices[id_]

iv_call = bs.implied_volatility(
    option_price=price,
    S=S,
    K=K,
    r=r,
    tau=tau,
    option_type=option_type,
    verbose=1,
)
print(f"Implied Volatility: {iv_call:.2%}")

Optimized Successfully!
Implied Volatility: 31.75%


Let's calculate the implied volatility for a Put option too.

In [9]:
id_ = 1
S = spot_price
K = strikes[id_]
r = 0.0375
tau = (
    datetime.datetime.strptime(expiry_date, "%Y-%m-%d")
    - datetime.datetime.strptime(today, "%Y-%m-%d")
).days / 365
option_type = "put"
price = put_prices[id_]

iv_put = bs.implied_volatility(
    option_price=price,
    S=S,
    K=K,
    r=r,
    tau=tau,
    option_type=option_type,
    verbose=1,
)
print(f"Implied Volatility: {iv_put:.2%}")

Optimized Successfully!
Implied Volatility: 37.72%


# `Option`

Now that we know the implied volatility of the option chain, we can calculate the option prices and greeks. We'll use implied volatility of 0.35.

## Calls

### Call(110)

In [10]:
iv = 0.35

call = Call(strike_price=110, expiry_date=expiry_date)
call

Call(110, 2023-04-21)

In [11]:
price, greeks = call.calculate_price(
    spot_price=spot_price,
    risk_free_rate=r,
    volatility=iv,
    day=today,
    return_greeks=True,
)

In [12]:
print(f"Price: {price:.2f}")
print(f"Greeks: {greeks}")

Price: 22.83
Greeks: {'delta': 0.84911185982313, 'gamma': 0.009712870562372489, 'theta': -13.2664032274749, 'vega': 16.117734861610362, 'rho': 24.61510270257206}


### Call(130)

In [13]:
iv = 0.35

call = Call(strike_price=130, expiry_date=expiry_date)
call

Call(130, 2023-04-21)

In [14]:
price, greeks = call.calculate_price(
    spot_price=spot_price,
    risk_free_rate=r,
    volatility=iv,
    day=today,
    return_greeks=True,
)

In [15]:
print(f"Price: {price:.2f}")
print(f"Greeks: {greeks}")

Price: 10.07
Greeks: {'delta': 0.5533517767222526, 'gamma': 0.0164055551176281, 'theta': -19.194885578746476, 'vega': 27.223711666438266, 'rho': 17.399717412297043}


## Puts

### Put(110)

In [16]:
iv = 0.35

put = Put(strike_price=110, expiry_date=expiry_date)
put

Put(110, 2023-04-21)

In [17]:
price, greeks = put.calculate_price(
    spot_price=spot_price,
    risk_free_rate=r,
    volatility=iv,
    day=today,
    return_greeks=True,
)

In [18]:
print(f"Price: {price:.2f}")
print(f"Greeks: {greeks}")

Price: 2.06
Greeks: {'delta': -0.15088814017687002, 'gamma': 0.009712870562372489, 'theta': -9.184824616633813, 'vega': 16.117734861610362, 'rho': -6.099242277638501}


### Put(130)

In [19]:
iv = 0.35

put = Put(strike_price=130, expiry_date=expiry_date)
put

Put(130, 2023-04-21)

In [20]:
price, greeks = put.calculate_price(
    spot_price=spot_price,
    risk_free_rate=r,
    volatility=iv,
    day=today,
    return_greeks=True,
)

In [21]:
print(f"Price: {price:.2f}")
print(f"Greeks: {greeks}")

Price: 9.08
Greeks: {'delta': -0.44664822327774745, 'gamma': 0.0164055551176281, 'theta': -14.371201765934282, 'vega': 27.223711666438266, 'rho': -18.899053927951794}


# `PayoffChart`

## Working with `PayoffChart`

In [22]:
K = 120
expiry_date2 = "2023-06-21"
c = Call(K, expiry_date)
p = Put(K, expiry_date)
c2 = Call(K+10, expiry_date2)
p2 = Put(K+10, expiry_date2)
c, p, c2, p2

(Call(120, 2023-04-21),
 Put(120, 2023-04-21),
 Call(130, 2023-06-21),
 Put(130, 2023-06-21))

In [23]:
[iv]*4

[0.35, 0.35, 0.35, 0.35]

In [28]:
positions = ["short", "short", "long", "long"]
sigmas = [iv]*4
options = [c, p, c2, p2]

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

In [29]:
fig = pc.payoff_chart(date=today, range=0.25)

In [30]:
fig = pc.payoff_chart(date=expiry_date, range=0.25)

## Some Strategies

In [40]:
prices = np.arange(100, 150, 5)
options_dict = {}
for price in prices:
    options_dict[f"c_{price}"] = Call(price, expiry_date)
    options_dict[f"p_{price}"] = Put(price, expiry_date)

options_dict

{'c_100': Call(100, 2023-04-21),
 'p_100': Put(100, 2023-04-21),
 'c_105': Call(105, 2023-04-21),
 'p_105': Put(105, 2023-04-21),
 'c_110': Call(110, 2023-04-21),
 'p_110': Put(110, 2023-04-21),
 'c_115': Call(115, 2023-04-21),
 'p_115': Put(115, 2023-04-21),
 'c_120': Call(120, 2023-04-21),
 'p_120': Put(120, 2023-04-21),
 'c_125': Call(125, 2023-04-21),
 'p_125': Put(125, 2023-04-21),
 'c_130': Call(130, 2023-04-21),
 'p_130': Put(130, 2023-04-21),
 'c_135': Call(135, 2023-04-21),
 'p_135': Put(135, 2023-04-21),
 'c_140': Call(140, 2023-04-21),
 'p_140': Put(140, 2023-04-21),
 'c_145': Call(145, 2023-04-21),
 'p_145': Put(145, 2023-04-21)}

In [43]:
mid_date = "2023-02-21"

### Long Call

Long call is a strategy where you buy a call option.

You use a long call if you think the stock price will increase.

In [45]:
options = [options_dict["c_130"]]
positions = ["long"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

### Short Call

In a short call strategy, you sell a call option. This is a bearish strategy.

In [46]:
options = [options_dict["c_130"]]
positions = ["short"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

### Long Put

This is a bearish strategy where you buy a put option.

In [49]:
options = [options_dict["p_130"]]
positions = ["long"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

### Short Put

This is a bullish strategy where you sell a put option.

In [48]:
options = [options_dict["p_130"]]
positions = ["short"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

### Straddle

An options straddle involves buying (or selling) both a call and a put with the same strike price and expiration on the same underlying asset.

#### Long Straddle

A long straddle is specially designed to assist a trader to catch profits no matter where the market decides to go. By purchasing a put and a call, the trader is able to catch the market's move regardless of its direction. If the market moves up, the call is there; if the market moves down, the put is there.

In [50]:
options = [options_dict["c_130"], options_dict["p_130"]]
positions = ["long", "long"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

#### Short Straddle

A short straddle is a strategy that involves selling both a call and a put with the same strike price and expiration on the same underlying asset. The strategy is designed to profit from a decrease in volatility.

In [51]:
options = [options_dict["c_130"], options_dict["p_130"]]
positions = ["short", "short"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

### Strangle

A strangle is an options strategy in which the investor holds a position in both a call and a put option with different strike prices, but with the same expiration date and underlying asset. A strangle is a good strategy if you think the underlying security will experience a large price movement in the near future but are unsure of the direction. However, it is profitable mainly if the asset does swing sharply in price.

This is similar to a straddle, except that the strike prices of the call and put are different.

#### Long Strangle

In [53]:
options = [options_dict["c_140"], options_dict["p_120"]]
positions = ["long", "long"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

#### Short Strangle

In [54]:
options = [options_dict["c_140"], options_dict["p_120"]]
positions = ["short", "short"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

### Butterfly Spread

This is a neutral strategy that uses four options contracts with the same expiration but three different strike prices:

- A higher strike price
- An at-the-money strike price
- A lower strike price


There are variuos types of butterfly spreads. We'll be using the Long Butterfly Spread.

#### Long Butterfly Spread

The long butterfly call spread is created by buying one in-the-money call option with a low strike price, writing two at-the-money call options, and buying one out-of-the-money call option with a higher strike price. The maximum profit is achieved if the price of the underlying at expiration is the same as the written calls. The max profit is equal to the strike of the written option, less the strike of the lower call, premiums, and commissions paid. The maximum loss is the initial cost of the premiums paid, plus commissions.

In [56]:
#Create a long butterfly spread
options = [options_dict["c_115"], options_dict["c_130"], options_dict["c_130"], options_dict["c_145"]]
positions = ["long", "short", "short", "long"]
sigmas = [iv]*len(options)

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

fig = pc.payoff_chart(date=mid_date, range=0.25)

### Iron Condor

An iron condor is an options strategy consisting of two puts (one long and one short) and two calls (one long and one short), and four strike prices, all with the same expiration date. The iron condor earns the maximum profit when the underlying asset closes between the middle strike prices at expiration. In other words, the goal is to profit from low volatility in the underlying asset.

The construction of the strategy is as follows:

1. Buy one out of the money (OTM) put with a strike price below the current price of the underlying asset. This OTM put option will protect against a significant downside move to the underlying asset. 
2. Sell one OTM or at the money (ATM) put with a strike price closer to the current price of the underlying asset.
3. Sell one OTM or ATM call with a strike price above the current price of the underlying asset.
4. Buy one OTM call with a strike price further above the current price of the underlying asset. This OTM call option will protect against a substantial upside move.


In [32]:
spot_price

129.6199951171875

In [34]:
pe1 = Put(115, expiry_date)
pe2 = Put(125, expiry_date)
ce1 = Call(135, expiry_date)
ce2 = Call(145, expiry_date)

positions = ["long", "short", "short", "long"]
options = [pe1, pe2, ce1, ce2]

In [35]:
sigmas = [iv]*4

pc = PayoffChart(
    options=options,
    positions=positions,
    sigmas=sigmas,
    spot_price=spot_price,
)

pc

PayoffChart([Put(115, 2023-04-21), Put(125, 2023-04-21), Call(135, 2023-04-21), Call(145, 2023-04-21)], ['long', 'short', 'short', 'long'], 129.6199951171875)

In [36]:
fig = pc.payoff_chart(date=expiry_date, range=0.25)